In [1]:
import json, shutil
import os, re, sys
from pathlib import Path
import glob, time


import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import numpy as np

import pandas as pd

from tqdm import tqdm
from tqdm import notebook
notebook.tqdm.pandas()
tqdm.pandas()

import psutil
from pyxtension.streams import stream


import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20, 12)


import spreprocessers as r
import spt_model as m



import torch as t
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import GroupShuffleSplit


from copy import deepcopy

print('__CUDA VERSION:', t.version.cuda)
print('__CUDNN VERSION:', t.backends.cudnn.version())
print(t.cuda.is_available())
if t.cuda.is_available():
    print(t.cuda.empty_cache())
    print(t.cuda.memory_summary(device=None, abbreviated=False))

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\yazee\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\yazee\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


__CUDA VERSION: 11.5
__CUDNN VERSION: 8302
True
None
|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |
|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |
|       fro

In [2]:
RANDOM_SEED = 42
PROCESSORS_COUNT = psutil.cpu_count(logical=False)
np.random.seed(RANDOM_SEED)


data_dir = Path('.')
train_dir = os.path.join(data_dir, 'train')
models_dir = os.path.join(data_dir, 'pt_models')
orders_path = os.path.join(data_dir, 'train_orders.csv')
ancestors_path = os.path.join(data_dir, 'train_ancestors.csv')


count = len(list(glob.iglob(os.path.join(train_dir, '*.json'))))
LOAD_NUM = int(count * 0.002) + 1
MULTI = PROCESSORS_COUNT * int(str(LOAD_NUM) [:-2])

print(f"\033[94mNumber of notebooks present in train set  = ", count)
print(f"\033[94mNumber of notebooks contribute in training = ", LOAD_NUM)

Number of notebooks present in train set  =  139256
Number of notebooks contribute in training =  279


In [3]:
start_time = time.time()
df = r.read_all_notebooks_(train_dir, LOAD_NUM, PROCESSORS_COUNT)

df = pd.concat(stream(np.array_split(df, PROCESSORS_COUNT)).mpmap(r.extract_features))

display(df.head())

print('-' * 125)

# Read Ordering data
df_orders = pd.read_csv(
    orders_path,
    index_col='id',
)
df_orders['cell_order'] = df_orders['cell_order'].str.split()  # Split the string representation of cell_ids into a list
df_orders = df_orders.squeeze(axis=1)


# build ranks as integers 
df = df.join(r.build_ranks_(df_orders, df, PROCESSORS_COUNT))


# Read Ancestors data
df = df.reset_index().merge(pd.read_csv(ancestors_path,  index_col='id'), on=["id"])

# convert integer ranks to percentages 
df["rank"] = df["rank"] / df.groupby("id")["cell_id"].transform("count")

display(df.head())

rduration = time.time() - start_time
print(f"Duration {rduration} seconds")

Train NBs: 100%|██████████| 279/279 [00:02<00:00, 100.31it/s]


<class 'pandas.core.frame.DataFrame'>
MultiIndex: 13633 entries, ('00001756c60be8', '1862f0a6') to ('0087b2bc5ba42e', 'f0b342ad')
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   cell_type  13633 non-null  category
 1   source     13633 non-null  object  
dtypes: category(1), object(1)
memory usage: 806.1+ KB
None


cell_type  \
id             cell_id              
00001756c60be8 1862f0a6      code   
               2a9e43d6      code   
               038b763d      code   
               2eefe0ef      code   
               0beab1cd      code   

                                                                    source  \
id             cell_id                                                       
00001756c60be8 1862f0a6  #  This Python 3 environment comes with many h...   
               2a9e43d6  import numpy as np\nimport pandas as pd\nimpor...   
               038b763d  import warnings\nwarnings.filterwarnings('igno...   
               2eefe0ef      matplotlib.rcParams.update({'font.size': 14})   
               0beab1cd  def evaluate_preds(train_true_values, train_pr...   

                         n_code_cells  n_markdown_cells  words_count  \
id             cell_id                                                 
00001756c60be8 1862f0a6      0.517241          0.482759          140   
               2a9e43d6      0.517241          0.482759           55   
               038b763d      0.517241          0.482759            3   
               2eefe0ef      0.517241          0.482759            2   
               0beab1cd      0.517241          0.482759           39   

                         letters_count  empty_lines_count  \
id             cell_id                                      
00001756c60be8 1862f0a6            930           0.235294   
               2a9e43d6            498           0.176471   
               038b763d             49           0.000000   
               2eefe0ef             45           0.000000   
               0beab1cd            694           0.052632   

                         comment_lines_count  full_lines_count  \
id             cell_id                                           
00001756c60be8 1862f0a6             0.411765          0.352941   
               2a9e43d6             0.000000          0.823529   
               038b763d             0.000000          1.000000   
               2eefe0ef             0.000000          1.000000   
               0beab1cd             0.000000          0.947368   

                         text_lines_count  tag_lines_count  
id             cell_id                                      
00001756c60be8 1862f0a6               0.0              0.0  
               2a9e43d6               0.0              0.0  
               038b763d               0.0              0.0  
               2eefe0ef               0.0              0.0  
               0beab1cd               0.0              0.0

-----------------------------------------------------------------------------------------------------------------------------


100%|██████████| 279/279 [00:00<00:00, 46834.66it/s]


Pandas Apply:   0%|          | 0/3 [00:00<?, ?it/s]

,id,cell_id,cell_type,source,n_code_cells,n_markdown_cells,words_count,letters_count,empty_lines_count,comment_lines_count,full_lines_count,text_lines_count,tag_lines_count,rank,ancestor_id,parent_id
0,00001756c60be8,1862f0a6,code,# This Python 3 environment comes with many h...,0.517241,0.482759,140,930,0.235294,0.411765,0.352941,0.0,0.0,0.000000,945aea18,NaN
1,00001756c60be8,2a9e43d6,code,import numpy as np\nimport pandas as pd\nimpor...,0.517241,0.482759,55,498,0.176471,0.000000,0.823529,0.0,0.0,0.034483,945aea18,NaN
2,00001756c60be8,038b763d,code,import warnings\nwarnings.filterwarnings('igno...,0.517241,0.482759,3,49,0.000000,0.000000,1.000000,0.0,0.0,0.068966,945aea18,NaN
3,00001756c60be8,2eefe0ef,code,matplotlib.rcParams.update({'font.size': 14}),0.517241,0.482759,2,45,0.000000,0.000000,1.000000,0.0,0.0,0.103448,945aea18,NaN
4,00001756c60be8,0beab1cd,code,"def evaluate_preds(train_true_values, train_pr...",0.517241,0.482759,39,694,0.052632,0.000000,0.947368,0.0,0.0,0.137931,945aea18,NaN


Duration 13.185547828674316 seconds


In [4]:
# for percentages 
INTREST_PERCENT = 0.95
CODE_TYPE = 'code'
MKDN_TYPE = 'markdown'
VALIDATION_RATIO = 0.15

NOT_GENERATED_COLUMNS = ['id', 'cell_id', 'source', 'cell_type', 'rank', 'ancestor_id', 'parent_id', ]
MODEL_USELESS = ['id', 'cell_type', 'ancestor_id', 'parent_id', ]
GENERATED_COLUMNS_COUNT = len(df.drop(['id', 'cell_id', 'source', 'cell_type', 'rank', 'ancestor_id', 'parent_id', ], axis=1).columns)

# BERT_MODEL_NAME = "microsoft/codebert-base"
BERT_MODEL_NAME = 'distilbert-base-uncased'
# BERT_MODEL_NAME = "microsoft/graphcodebert-base"

# OPTIMIZER = 'adam'
OPTIMIZER = 'nadam'

# MAX_LENGTH = int(df[df.cell_type == CODE_TYPE].words_count.quantile(INTREST_PERCENT)) # max length our sequences will be (e.g. how many words from a Tweet does our model see?)
MAX_LENGTH = 196

ACCUMULATION_SETPS = 2
BATCH_SIZE = 8
EPOCHS = 1
TOTAL_MAX_LEN = 285

In [5]:
splitter = GroupShuffleSplit(n_splits=1, test_size=VALIDATION_RATIO, random_state=RANDOM_SEED)

# Split, keeping notebooks with a common origin (ancestor_id) together
ids_train, ids_valid = next(splitter.split(df, groups=df["ancestor_id"]))
print(ids_train[:5], ids_valid[:5])

# extract code cells for each notebook
df_train = df.loc[ids_train, :].reset_index(drop=True)
df_valid = df.loc[ids_valid, :].reset_index(drop=True)
display(df_train.head(2))
display(df_valid.head(2))

# dict_cellid_source = dict(zip(df['cell_id'].values, df['source'].values))

print(df_train.shape, df_valid.shape) #, len(dict_cellid_source))

train_triplets = []
val_triplets = []

for lst in list(stream(np.array_split(df_train, PROCESSORS_COUNT)).mpmap(r.generate_triplet)):
    train_triplets.extend(lst)

for lst in list(stream(np.array_split(df_valid, PROCESSORS_COUNT)).mpmap(r.generate_triplet)):
    val_triplets.extend(lst)

print(len(train_triplets), len(val_triplets))

[0 1 2 3 4] [905 906 907 908 909]


,id,cell_id,cell_type,source,n_code_cells,n_markdown_cells,words_count,letters_count,empty_lines_count,comment_lines_count,full_lines_count,text_lines_count,tag_lines_count,rank,ancestor_id,parent_id
0,00001756c60be8,1862f0a6,code,# This Python 3 environment comes with many h...,0.517241,0.482759,140,930,0.235294,0.411765,0.352941,0.0,0.0,0.000000,945aea18,NaN
1,00001756c60be8,2a9e43d6,code,import numpy as np\nimport pandas as pd\nimpor...,0.517241,0.482759,55,498,0.176471,0.000000,0.823529,0.0,0.0,0.034483,945aea18,NaN


,id,cell_id,cell_type,source,n_code_cells,n_markdown_cells,words_count,letters_count,empty_lines_count,comment_lines_count,full_lines_count,text_lines_count,tag_lines_count,rank,ancestor_id,parent_id
0,0008ba887b3817,006235ba,code,import numpy as np\nimport pandas as pd\nimpor...,0.530612,0.469388,95,881,0.111111,0.037037,0.851852,0.0,0.0,0.081633,bde84293,ed1c7969a2900a
1,0008ba887b3817,f643db5c,code,"train=pd.read_csv(""/kaggle/input/covid19-globa...",0.530612,0.469388,3,168,0.000000,0.000000,1.000000,0.0,0.0,0.122449,bde84293,ed1c7969a2900a


(11107, 16) (2526, 16)
24923 6221


In [6]:
train_ds = m.BDataset(
    df_train, 
    triplts=train_triplets,
    max_len=MAX_LENGTH, 
    bert_model_name=BERT_MODEL_NAME, 
    total_max_len=TOTAL_MAX_LEN, 
    drop=MODEL_USELESS
)
val_ds = m.BDataset(
    df_valid, 
    triplts=val_triplets,
    max_len=MAX_LENGTH, 
    bert_model_name=BERT_MODEL_NAME, 
    total_max_len=TOTAL_MAX_LEN,  
    drop=MODEL_USELESS
)
print(train_ds[0], val_ds[0])

c_row:  [1 '2a9e43d6'
 'import numpy as np\nimport pandas as pd\nimport random\nfrom sklearn.model_selection import train_test_split, cross_val_score\nfrom sklearn.preprocessing import StandardScaler, RobustScaler\nfrom catboost import CatBoostRegressor\nfrom sklearn.ensemble import RandomForestRegressor\nfrom sklearn.metrics import r2_score as r2\nfrom sklearn.model_selection import KFold, GridSearchCV\nfrom datetime import datetime\nimport matplotlib\nimport matplotlib.pyplot as plt\nimport seaborn as sns\n%matplotlib inline'
 0.5172413793103449 0.4827586206896552 55 498 0.17647058823529413 0.0
 0.8235294117647058 0.0 0.0 0.034482758620689655]
m_row:  [30 '21616367' '*деление признаков на числовые и текстовые*'
 0.5172413793103449 0.4827586206896552 6 43 0.0 0.0 1.0 0.0 1.0
 0.29310344827586204]
c_row:  [1 'f643db5c'
 'train=pd.read_csv("/kaggle/input/covid19-global-forecasting-week-5/train.csv")\ntest=pd.read_csv("/kaggle/input/covid19-global-forecasting-week-5/test.csv")\ntrain.hea

In [8]:
# build code DataLoader
train_loader = DataLoader(
    train_ds, 
    batch_size=BATCH_SIZE, 
    shuffle=True, 
    num_workers=PROCESSORS_COUNT, 
    pin_memory=False, 
    drop_last=True
)
val_loader = DataLoader(
    val_ds, 
    batch_size=BATCH_SIZE, 
    shuffle=False, 
    num_workers=PROCESSORS_COUNT, 
    pin_memory=False, 
    drop_last=False
)
print(train_loader, val_loader)

<torch.utils.data.dataloader.DataLoader object at 0x000001A10C162CE0> <torch.utils.data.dataloader.DataLoader object at 0x000001A10C162E90>


In [7]:
raise

RuntimeError: No active exception to reraise

In [9]:
########################################################################################################################
model = m.BModel(
    BERT_MODEL_NAME, 
    GENERATED_COLUMNS_COUNT * 2
).cuda()

model, y_pred = m.train(
    model, 
    train_loader, 
    val_loader, 
    epochs=EPOCHS, 
    accumulation_steps=ACCUMULATION_SETPS, 
    model_name=BERT_MODEL_NAME,
    opt=OPTIMIZER, 
    path=os.path.join(models_dir, 'single_bert_checkpoint.pt')
)

df_valid["pred"] = df_valid.groupby(["id", "cell_type"])["rank"].rank(pct=True)
df_valid["pred"] = y_pred
########################################################################################################################

y_dummy = df_valid.sort_values("pred").groupby('id')['cell_id'].apply(list)
print('Final accuracy for code is:', r.kendall_tau(df_orders.loc[y_dummy.index], y_dummy))

# best_model_state = deepcopy(code_model.state_dict())
# t.save(best_model_state, f'./pt_models/code_model_state_dict.pt')
# t.save(code_model, f'./pt_models/code_model.pt')

rduration = time.time() - start_time
print(f"\nDuration {rduration} seconds")

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


  0%|          | 0/3115 [00:29<?, ?it/s]


RuntimeError: Caught UnicodeEncodeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "C:\Users\yazee\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\data\_utils\worker.py", line 287, in _worker_loop
    data = fetcher.fetch(index)
  File "C:\Users\yazee\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\data\_utils\fetch.py", line 49, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "C:\Users\yazee\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\data\_utils\fetch.py", line 49, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "E:\AI4Code\spt_model.py", line 331, in __getitem__
    print('m_row: ', m_row)
  File "C:\Users\yazee\AppData\Local\Programs\Python\Python310\lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
UnicodeEncodeError: 'charmap' codec can't

In [ ]:
raise

In [ ]:
test_dir = os.path.join(data_dir, 'test')
print(f"\033[94mNumber of notebooks present in test set  = ", len(list(glob.iglob(os.path.join(test_dir, '*.json')))))

df_test = r.read_all_notebooks_(test_dir, 4, 2, desc="Tests NBs")

df_test = r.extract_features(df_test).reset_index()
df_test['rank'] = 0

MODEL_USELESS = ['id', 'cell_id', 'cell_type', ]

BERT_MODEL_NAME = 'distilbert-base-uncased'
PROCESSORS_COUNT = psutil.cpu_count(logical=False)

BATCH_SIZE = 8
TOTAL_MAX_LEN = 256
MAX_LENGTH = 128

df_test['rank'] = m.predict(
    model_path=BERT_MODEL_NAME,
    check_point=os.path.join(models_dir, 'single_bert_checkpoint.pt'), 
    batch_size=BATCH_SIZE, 
    num_workers=PROCESSORS_COUNT, 
    max_len=MAX_LENGTH, 
    total_max_len=TOTAL_MAX_LEN, 
    data=df_test, 
    drop=MODEL_USELESS
)

df_test = df_test.sort_values("rank").groupby("id")["cell_id"].apply(lambda x: " ".join(x)).reset_index()
df_test.rename(columns={"cell_id": "cell_order"}, inplace=True)
display(df_test.head())

# df_test.to_csv("submission.csv", index=False)